In [150]:
# import packages
import pandas as pd
import YouTubeAPI 
import time
import numpy as np
import jieba
import re

In [2]:
# 阿左的code

# vars
cURL="https://www.youtube.com/channel/"
vURL="https://www.youtube.com/watch?v="
CreateTime=time.strftime("%y%m%d%H%M",time.localtime())

def SearchResult(Result):
    Data={}
    for SR in Result:
        cId=SR["snippet"]["channelId"]
        vId=SR["id"]["videoId"]
        if(cId not in Data):
            Data[cId]=[]
        Data[cId].append(vId)
    return Data
def SearchDetail(SR):
    SD={}
    #CreateTime=time.strftime("%y%m%d%H%M",time.localtime())
    for cId in SR:
        cD=YouTubeAPI.YouTubeChannel_by_cId(cId)
        if cD==[]:continue
        cD=cD[0]
        cInfo=cD["snippet"]
        if "country" not in cInfo:
#             print("-"*50)
#             print(cInfo)
#             print("-"*50)
            continue
#         print(cInfo["country"])
        if cInfo["country"]!="TW":continue
        cNm=cInfo["title"]
        SD[cNm]={}
        cData=SD[cNm]
        cData["channelTitle"]=cNm
        cData["channelId"]=cId
        cData["channelUrl"]=cURL+cId
        if "customUrl" in cInfo:cData["customID"]=cInfo["customUrl"]
        cData["description"]=cInfo["description"]
        cData["publishedAt"]=cInfo["publishedAt"]
        cData["cCount"]={}
        cData["cCount"][CreateTime]=cD["statistics"]
        cData["Video"]={}
        for vId in SR[cId]:
            vD=YouTubeAPI.YouTubeVideo_by_vId(vId)[0]
            vInfo=vD["snippet"]
            vNm=vInfo["title"]
            cData["Video"][vNm]={}
            vData=cData["Video"][vNm]
            vData["videoTitle"]=vNm
            vData["videoId"]=vId
            vData["videoUrl"]=vURL+vId
            vData["description"]=vInfo["description"]
            vData["publishedAt"]=vInfo["publishedAt"]
            vData["tags"]=vInfo["tags"] if ("tags" in vInfo) else "Null"
            vData["vCount"]={}
            vData["vCount"][CreateTime]=vD["statistics"]
    return SD

In [279]:
# collect data from api
words = ['口紅','春節彩妝','粉餅','彩妝','遮瑕膏']
keyword = words[3]

token = None

chanel_info_excel = pd.DataFrame()
video_all_df_excel = pd.DataFrame()
all_df_excel = pd.DataFrame()

for i in range(10):
    print(i)
    top50_results = YouTubeAPI.YouTubeSearch(keyword,50,token)
    # collect channel_id and video_id from results and recollect more data by search_channel & search_video
    chanel_id_video_id = SearchResult(top50_results[1])
    more_datail = SearchDetail(chanel_id_video_id)


    # json to df and rename
    trans_df = pd.DataFrame(more_datail).T

    trans_df = trans_df.reset_index(drop=True)
    # chose chanel used info
    chanel_info = trans_df[['channelId','customID','channelTitle','publishedAt','description']].copy()

    chanel_info.columns = ['頻道ID','作者ID','頻道名稱','頻道建立日期','頻道敘述']

    # 把json從rows拉出來變成df，給予新的欄位名稱
    chanel_count = trans_df[['cCount', 'channelId']].copy()

    cCount_df = pd.concat(chanel_count['cCount'].apply(lambda x: pd.DataFrame(x).T.reset_index()).values,ignore_index=True)

    cCount_info = chanel_count[['channelId']].join(cCount_df)

    cCount_info_used = cCount_info[['channelId','index','viewCount','videoCount','subscriberCount']].copy()

    cCount_info_used.columns = ['頻道ID','最後更新日期','最新觀看次數','最新影片數量','最新訂閱次數']

    # join 回原本的df
    chanel_info_v1 = chanel_info.merge(cCount_info_used,on='頻道ID')

    # 重新排序    channel資訊成功 
    chanel_info_v2 = chanel_info_v1[['頻道ID', '作者ID', '頻道名稱', '頻道建立日期','最新觀看次數', '最新影片數量',
           '最新訂閱次數','最後更新日期','頻道敘述']].copy()

    ### 影片資訊 
    # 把json從rows拉出來變成df，給予新的欄位名稱
    video_info = trans_df[['channelTitle','Video']].copy()

    video_all_df = pd.DataFrame()
    for channelTitle in video_info['channelTitle']:
        print(channelTitle)
        videos = video_info.loc[video_info['channelTitle'] == channelTitle,['Video']]

        video_df = pd.DataFrame(videos['Video'].values[0]).T.reset_index(drop=True)

        # 把json從rows拉出來變成df，給予新的欄位名稱  "vCount"
        video_count = video_df[['vCount', 'videoId']].copy()

        vCount_df = pd.concat(video_count['vCount'].apply(lambda x: pd.DataFrame(x).T.reset_index()).values,ignore_index=True,sort=False)

        vCount_info = video_count[['videoId']].join(vCount_df)

        video_df['tags'] = video_df['tags'].apply(lambda x: ','.join(x))[0]

        video_df_all_info = video_df.merge(vCount_info,on=['videoId'])
        # check like/dilike
        if video_df_all_info.columns.isin(['likeCount']).any():
            pass
        else:
            video_df_all_info['likeCount'] = np.nan
        if video_df_all_info.columns.isin(['dislikeCount']).any():
            pass
        else:
            video_df_all_info['dislikeCount'] = np.nan
        if video_df_all_info.columns.isin(['commentCount']).any():
            pass
        else:
            video_df_all_info['commentCount'] = np.nan
        # sort columns
        sort_col = ['videoId','videoTitle','publishedAt','viewCount','likeCount','dislikeCount'
                    ,'commentCount','index','description','tags']
        video_df_sort_col = video_df_all_info[sort_col].copy()
        # alter colname
        zh_n = ['影片ID','影片名稱','影片建立日期','最新觀看次數','最新喜歡數','最新不喜歡數','最新留言數','最後更新日期','影片敘述','標籤']
        video_df_sort_col.columns = zh_n

        video_df_sort_col.insert(0,'頻道名稱',channelTitle)
        video_all_df = video_all_df.append(video_df_sort_col,ignore_index=True,sort=False)

    # for jieba
    # 載入停用詞

    stops = []
    with open('stop_word_zh.txt','r') as r_txt:
        for i in r_txt.readlines():
            stops.append(i.strip())

    video_all_df_c = video_all_df[['頻道名稱','影片ID','影片敘述']].copy()

    all_df = pd.DataFrame()
    for video_id in video_all_df_c['影片ID']:
        series = video_all_df_c.loc[video_all_df_c['影片ID']==video_id,'影片敘述']
        temp = []
        for i in jieba.cut(series.values[0]): 
            if re.search(u'^[\u4e00-\u9fa5]+$', i): 
                if i not in stops:
                    temp.append(i)
        tdf = pd.DataFrame({'敘述結巴':temp})
        tdf['影片ID'] = video_id
        all_df = all_df.append(tdf,ignore_index=True)

    all_df = all_df.merge(video_all_df_c[['頻道名稱','影片ID']],on=['影片ID'])   

    chanel_info_excel = chanel_info_excel.append(chanel_info_v2,ignore_index=True)
    video_all_df_excel = video_all_df_excel.append(video_all_df,ignore_index=True)
    all_df_excel = all_df_excel.append(all_df,ignore_index=True)

    token = top50_results[0]
    
    if token == 'last_page':
        break
        
writer = pd.ExcelWriter('查找結果統計_{}.xlsx'.format(keyword), engine='xlsxwriter')

chanel_info_excel = chanel_info_excel.drop_duplicates(['頻道ID'])
video_all_df_excel = video_all_df_excel.drop_duplicates(['影片ID']) 

chanel_info_excel.to_excel(writer, sheet_name='頻道資訊')
video_all_df_excel.to_excel(writer, sheet_name='影片資訊')
all_df_excel.to_excel(writer, sheet_name='結巴敘述')

writer.save()

0
GINA HELLO!
百變沛莉 Peri
MONROE夢露
居妮Ginny Daily
女人我最大
LILLIAN.M
Mii黃小米
Hello Catie
葛瑞絲Gracee
Kevin老師的時尚美妝頻道
The Regina.
I'm Charlie
iamPhoebeLiu
Cindy H
陳彥婷
1
Mii黃小米
Wei！ 威蒿小宇宙有什麼
OL煮狐狸日常
Kevin老師的時尚美妝頻道
百變沛莉 Peri
居妮Ginny Daily
NO FAN NO FUN
HANA西村英惠
NEH TV
艾米莉豬Emily
Hello Catie
游絲棋Szu chi Yu
TingLin婷
年代新聞CH50
GINA HELLO!
余星星Yuki
Yoni Lee李妍熹
2
Celeste Wu 大沛
Mii黃小米
百變沛莉 Peri
I'm Charlie
Hello Catie
Yoni Lee李妍熹
Sh!a up
tramy崔咪
SHOPAHOLIC凱利依法
NO FAN NO FUN
SETN三立新聞網
年代新聞CH50
KAITV
VOGUE Taiwan
NEH TV
MONROE夢露
3
小伶玩具 (Xiaoling Toys)
Cindy H
KAITV
NO FAN NO FUN
台灣蘋果日報
GINA HELLO!
Hello Catie
This is Stella
百變沛莉 Peri
VOGUE Taiwan
居妮Ginny Daily
一隻阿圓 I am CIRCLE
MONROE夢露
Mii黃小米
Bunny makeup soul邦邦彩妝魂
女人我最大
4
吉米老濕
百變沛莉 Peri
iamPhoebeLiu
I'm Charlie
找愛TV
居妮Ginny Daily
小石頭的家 JustRock
EVALIN
Sunny&Yummy的玩具箱kids toys
街頭時尚Astor
MONROE夢露
魚乾
ELLE TAIWAN
年代新聞CH50
tramy崔咪
NO FAN NO FUN
TVBS 優選頻道
Shu Uemura Taiwan
BeautYuhan
5
百變沛莉 Peri
Megan Zhang
Kevin老師的時尚美妝頻道
Hello Catie
女人我最大
Ksnancy
劉力穎 Liying Liu
GI